In [1]:
#### Linear regression model using nn.Linear 
### pytorch modelling exercise

In [3]:
import torch
import matplotlib.pyplot as plt
import numpy as np

print(torch.__version__)

1.13.0+cpu


In [5]:
###device agnostic 

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using device: {device}")

using device: cpu


In [12]:
### creating data frames
### atleast 100 data points

weight = 0.3
bias = 0.9

start, end, step = 0, 1, 0.005

x = torch.arange(start, end, step) #created 200 data points

200

In [16]:
#### creating training and test data (separately)
train_data = int(0.8 * len(x))

x_train, y_train = x[:train_data], x[:train_data]
x_test, y_test = x[train_data:], x[train_data:]

len(x_train), len(y_train), len(x_test), len(y_test)

(160, 160, 40, 40)